<a href="https://colab.research.google.com/github/mezenka/pandas-ds1/blob/main/pandas_ds1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Sales Analysis**

In [7]:
import pandas as pd
import os
import requests
from io import StringIO

##### **Merging multiple csv into single df**

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/mezenka/pandas-ds1/main/data/Sales_January_2019.csv')

df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,141234,iPhone,1,700,01/22/19 21:25,"944 Walnut St, Boston, MA 02215"
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,"185 Maple St, Portland, OR 97035"
2,141236,Wired Headphones,2,11.99,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016"
3,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,"738 10th St, Los Angeles, CA 90001"
4,141238,Wired Headphones,1,11.99,01/25/19 11:59,"387 10th St, Austin, TX 73301"


In [32]:
# Define the GitHub repository URL
github_repo_url = 'https://api.github.com/repos/mezenka/pandas-ds1/contents/data'

# Fetch the repository contents using GitHub API
response = requests.get(github_repo_url)

# Check if the request was successful and proceed if it was
if response.status_code == 200:
    # Parse the JSON response containing the repository contents
    data = response.json()

    # Filter the URLs to include only the CSV files
    csv_files = [file['name'] for file in data if file['name'].endswith('.csv')]

    # Print the list of CSV files (optional)
    #print("CSV Files:")
    print(csv_files)
else:
    print("Failed to fetch the repository contents from GitHub.")

# Continue with the rest of the code as before to load and concatenate the CSV files
# ...

['Sales_April_2019.csv', 'Sales_August_2019.csv', 'Sales_December_2019.csv', 'Sales_February_2019.csv', 'Sales_January_2019.csv', 'Sales_July_2019.csv', 'Sales_June_2019.csv', 'Sales_March_2019.csv', 'Sales_May_2019.csv', 'Sales_November_2019.csv', 'Sales_October_2019.csv', 'Sales_September_2019.csv']


In [37]:
# Defining the GitHub repository URL
github_repo_url = 'https://api.github.com/repos/mezenka/pandas-ds1/contents/data'

# Fetching the repository contents using GitHub API
response = requests.get(github_repo_url)

# Checking if the request was successful and proceeding if it was
if response.status_code == 200:
    # Parsing the JSON response containing the repository contents
    data = response.json()

    # Filtering the URLs to include only the CSV files
    csv_files = [file['name'] for file in data if file['name'].endswith('.csv')]

    dfs = []  # List to store DataFrames

for file_name in csv_files:
    url = github_url + file_name
    response = requests.get(url)

    if response.status_code == 200:
        content = response.text
        df = pd.read_csv(StringIO(content))
        dfs.append(df)
    else:
        print(f"Failed to fetch {file_name} from the GitHub repository.")

all_data = pd.concat(dfs, ignore_index=True)

all_data.head()


,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,NaN,NaN,NaN,NaN,NaN,NaN
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"


In [23]:
# the same with file names by hand
"""github_url = 'https://github.com/mezenka/pandas-ds1/raw/main/data/'

csv_files = ['Sales_January_2019.csv','Sales_February_2019.csv','Sales_March_2019.csv','Sales_April_2019.csv','Sales_May_2019.csv','Sales_June_2019.csv','Sales_July_2019.csv','Sales_August_2019.csv','Sales_September_2019.csv','Sales_October_2019.csv','Sales_November_2019.csv','Sales_December_2019.csv']

dfs = []

for file_name in csv_files:
    url = github_url + file_name
    response = requests.get(url)

    if response.status_code == 200:
        content = response.text
        df = pd.read_csv(StringIO(content))
        dfs.append(df)
    else:
        print(f"Failed to fetch {file_name} from the GitHub repository.")

concatenated_df = pd.concat(dfs, ignore_index=True)

concatenated_df.head()"""

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,141234,iPhone,1,700,01/22/19 21:25,"944 Walnut St, Boston, MA 02215"
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,"185 Maple St, Portland, OR 97035"
2,141236,Wired Headphones,2,11.99,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016"
3,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,"738 10th St, Los Angeles, CA 90001"
4,141238,Wired Headphones,1,11.99,01/25/19 11:59,"387 10th St, Austin, TX 73301"


In [ ]:
# cleaning


In [40]:
# adding columns for month and year
all_data['Month'] = all_data['Order Date'].str[:2]
all_data['Month'] = all_data['Month'].astype('int64')
all_data.head()

ValueError: ignored